In [ ]:
%matplotlib inline
from modsim import *

earth = Condition(thickness = 2414,
                     surface_area = 5.099686589e+14,
                     density = 5510)


earth.radius = sqrt(earth.surface_area / (8 * np.pi))
earth.volume1 = earth.thickness * earth.radius** 2 * np.pi
earth.mass = earth.volume1 * earth.density



baseball = Condition(x = 0, 
                      y = (earth.thickness/2) + 1,
                      ay = 9.8,
                      G = 6.67e-11,
                      mass = 145e-3,
                      diameter = 73e-3,
                      rho = 1.2,
                      C_d = 0.3,
                      angle = 45,
                      velocity = 40,
                      duration = 25.1)

def make_system(condition):
    """Make a system object.
    
    condition: Condition object with angle, velocity, x, y,
               diameter, duration, g, mass, rho, and C_d
               
    returns: System object
    """
    unpack(condition)
    
    # convert angle to degrees
    theta = np.deg2rad(angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, velocity)
    
    # make the initial state
    init = State(x=x, y=y, vx=vx, vy=vy)
    
    # compute area from diameter
    area = np.pi * (diameter/2)**2
    
    # compute timestamps
    ts = linspace(0, duration, 101)
    
    return System(init=init, G=G, mass=mass, 
                  area=area, rho=rho, C_d=C_d, ts=ts)

def slope_func(state, t, system):
    x, y, vx, vy = state
    unpack(system)
    p = Vector(x, y)
    v = Vector(vx, vy)
    f_grav = (-G * earth.mass * mass * p.hat()/ (p.mag**1))
    f_drag = -rho * v.mag * v * C_d * area / 2
    
    a_grav = f_grav / mass
    a_drag = f_drag / mass
    
    a_y= Vector(0, -ay)

    
    a = a_grav + a_drag + a_y
    
    print(a_drag-a_grav)
    
    return vx, vy, a.x, a.y
    


    

In [ ]:
system = make_system(baseball)
slope_func(system.init, 0, system)

run_odeint(system, slope_func)

xs = system.results.x
ys = system.results.y - (earth.thickness/2) + 1

xv = system.results.vx
yv = system.results.vy

newfig()
plot(xs[1:], label='x')
plot(ys[1:], label='y')

decorate(xlabel='Time (s)',
         ylabel='Position (m)')


In [ ]:
newfig()
decorate(xlabel='x position (m)',
         ylabel='y position (m)',
         xlim=[-30, 30],
         ylim=[-50, 15000],
         legend=False)

for x, y in zip(system.results.x, system.results.y):
    plot(x, y, 'bo', update=True)
    sleep(0.01)